### Multinomial NB on training and test data.

In [29]:
import pandas as pd

# loading 2 files, and adding headers to the first one, takes around 2 minutes
file1=pd.read_csv('https://raw.githubusercontent.com/gitjuli/DAV-5400/master/Data/Final%20Project%20Data/training.1600000.processed.noemoticon_0.csv', encoding = "ISO-8859-1")
file2=pd.read_csv('https://raw.githubusercontent.com/gitjuli/DAV-5400/master/Data/Final%20Project%20Data/training.1600000.processed.noemoticon_1.csv', encoding = "ISO-8859-1")
# adding column names to the 4 files
file1.columns = ['label','id','date','flag','user','text']
file2.columns = ['label','id','date','flag','user','text']

In [30]:
# appending all together
data=file1.append(file2,sort=False)#.append(file3,sort=False).append(file4,sort=False)

In [31]:
data=data.loc[:,['label', 'text']]
data.head(5)

,label,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [32]:
data['label'].value_counts()

0    799999
4    200001
Name: label, dtype: int64

In [33]:
# splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], random_state=1)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)

In [35]:
# fit
cv.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[a-zA-Z0-9]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>,
        vocabulary=None)

In [36]:
# transform training data
X_train_dtm = cv.transform(X_train)

In [37]:
# examine the document-term matrix
X_train_dtm

<750000x365603 sparse matrix of type '<class 'numpy.int64'>'
	with 5612948 stored elements in Compressed Sparse Row format>

In [38]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = cv.transform(X_test)
X_test_dtm

<250000x365603 sparse matrix of type '<class 'numpy.int64'>'
	with 1786851 stored elements in Compressed Sparse Row format>

In [39]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)

Wall time: 345 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)
y_pred_class[:5]

array([0, 4, 0, 0, 4], dtype=int64)

In [41]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.842636